In [9]:
import re
import nltk

## Tag the training data

In [40]:
def clean_sentence(s):
    c = s.lower().strip()
    return re.sub('[^a-z ]', '', c)

In [45]:
def translate_word_class(tag):
    if tag in ['JJ', 'JJR', 'JJS']:
        return 'adjective'
    if tag in ['NN', 'NNS', 'NNP', 'NNPS']:
        return 'noun'
    if tag in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']:
        return 'verb'
    if tag in ['CC', 'IN']:
        return 'link'
    return 'other'

In [51]:
def map_tagging(sentence):
    translation = list()
    for word, tag in sentence:
        translation.append((word, translate_word_class(tag)))
    return translation

In [41]:
train_sentences = list()
with open('./data_set/training_set70.txt') as train:
    for line in train:
        train_sentences.append(clean_sentence(line))

In [42]:
train_sentences[:5]

['correct', 'dick agreed', 'sorry wrong', 'where is mary', 'he is happy now']

In [43]:
tagged_sentences = list()
for sentence in train_sentences:
    words = nltk.word_tokenize(sentence)
    tagged_sentences.append(nltk.pos_tag(words))

In [44]:
tagged_sentences[:5]

[[('correct', 'NN')],
 [('dick', 'NNS'), ('agreed', 'VBD')],
 [('sorry', 'NN'), ('wrong', 'JJ')],
 [('where', 'WRB'), ('is', 'VBZ'), ('mary', 'JJ')],
 [('he', 'PRP'), ('is', 'VBZ'), ('happy', 'JJ'), ('now', 'RB')]]

In [53]:
my_tagged_sentences = list(map(map_tagging, tagged_sentences))

In [54]:
my_tagged_sentences[:5]

[[('correct', 'noun')],
 [('dick', 'noun'), ('agreed', 'verb')],
 [('sorry', 'noun'), ('wrong', 'adjective')],
 [('where', 'other'), ('is', 'verb'), ('mary', 'adjective')],
 [('he', 'other'), ('is', 'verb'), ('happy', 'adjective'), ('now', 'other')]]